# Importando Bibliotecas

In [110]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

# Obtendo e Tratando os Dados

In [111]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [112]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [113]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,POPR4,10.17,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"
1,MNSA4,0.47,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
2,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
3,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
4,CSTB4,147.69,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,BIDI4,25.53,3751.95,20.32,0.000,"0,03%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,54%",129353000.0,3.240140e+09,0.00,"30,91%"
956,BIDI11,77.00,3772.04,20.43,0.000,"0,03%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,54%",294646000.0,3.240140e+09,0.00,"30,91%"
957,BIDI3,26.15,3843.07,20.81,0.000,"0,03%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,54%",18222300.0,3.240140e+09,0.00,"30,91%"
958,NTCO3,60.80,5423.82,2.92,2.154,"0,00%",1.354,30.63,34.07,-5.45,37.09,17.37,"6,32%","0,01%",1.18,"5,05%","0,05%",361381000.0,2.869130e+10,0.49,"0,00%"


In [114]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

In [115]:
df.shape

(960, 21)

# Analizando os Dados

In [116]:
df = df[df['Liq.2meses'] > 1000000]
df.shape

(285, 21)

In [117]:
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [118]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,GPIV33,CMIN3
2,3,GOAU3,PSSA3
3,4,EVEN3,ODPV3
4,5,GOAU4,AURA33
...,...,...,...
145,146,ELET6,LINX3
146,147,EZTC3,ANIM3
147,148,LAME4,POWE3
148,149,INTB3,CYRE3


In [119]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [120]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [121]:
t = pd.concat([a,b])
t

,ABEV3,AGRO3,ALLD3,ALSO3,ALUP11,ANIM3,ASAI3,AURA33,B3SA3,BEEF3,BLAU3,BMOB3,BOBR4,BRDT3,BRFS3,BRKM5,BRML3,CAML3,CARD3,CCPR3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,COCE5,CPFE3,CPLE11,CPLE3,CPLE6,CRFB3,CRPG5,CSED3,CSMG3,CSNA3,CURY3,CYRE3,DEXP3,DIRR3,...,TESA3,TGMA3,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP3,UNIP6,USIM3,USIM5,VALE3,VIVT3,VLID3,VVAR3,WIZS3,WSON33,ALPA4,AMBP3,BOAS3,CASH3,ENEV3,ESPA3,GNDI3,HAPV3,LINX3,MATD3,MOSI3,NGRD3,ORVR3,PETZ3,RADL3,RDOR3,RENT3,SULA11,TFCO4,VAMO3,VIVA3,WEGE3
pos,119,61,78,150.0,9,135,81,21,103,23,132,92,88,105,87,24,140.0,54,95,85.0,93,104.0,120.0,43,28,13,133.0,55,18,14,20,69,91,144.0,32,36,74,108,99,50,...,82,107,56,124,73,16,89,127,40,39,35,34,10,90.0,138.0,84,29,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,55,54,36,NaN,43,147,23,5,29,25,8,27,58,101,120,26,NaN,69,48,NaN,100,NaN,NaN,94,95,2,NaN,78,103,104,105,81,20,NaN,79,18,35,149,88,142,...,123,61,118,19,93,53,134,144,11,12,44,45,13,NaN,NaN,96,1,46,40.0,113.0,112.0,59.0,139.0,145.0,121.0,97.0,146.0,28.0,9.0,91.0,130.0,102.0,115.0,137.0,77.0,114.0,32.0,84.0,76.0,22.0


In [122]:
rank = t.dropna(axis=1).sum()

In [125]:
rank.sort_values()[:15]


PSSA3      4
CMIN3     15
PLPL3     22
VALE3     23
MRFG3     23
AURA33    26
WIZS3     30
TASA3     37
TASA4     40
FHER3     40
BEEF3     48
BRKM5     50
UNIP3     51
UNIP6     51
ALUP11    52
dtype: int64